In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from utils import get_data, get_probabilities, estimate_q_Z_given_A, get_probabilities_one_hot
from sklearn.decomposition import NMF  # Placeholder for volmin factorization
from volmin_nmf import *
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from scipy.optimize import nnls

/Users/rickcollins64/opt/anaconda3/envs/MV00/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
# ======================================================================================================================
#                                                     Flags
# ======================================================================================================================
'''
debug: Flag to print debug information such as the shape of the data, examples of data, confusion matrices etc.
summary: Flag to print summary information such as accuracy of models and the final q(Y|a)
'''
debug = False
summary = False

# ======================================================================================================================
#                                           Data Generation Parameters
# ======================================================================================================================
'''
These parameters are used to generate the data.
p_source: Probability of a sample being from the source domain
p_target: Probability of a sample being from the target domain
total: Total number of samples
factorisation_atol: Absolute tolerance for the factorisation
specific_a_index: Index of the specific A value to be used
num_classes_Y: Number of classes for Y
num_classes_W: Number of classes for W
num_features_Z: Number of features for Z
num_features_A: Number of features for A
'''
p_target = 0.2
p_source = 1 - p_target
total = 10000
factorisation_atol = 1e-1
specific_a_index = 0  # First value of A
num_classes_Y = 2 
num_classes_W = 2
num_features_Z = 2
num_features_A = 3
num_epsilon = 2 # min(W_source.shape[1], Z_source.shape[1]) # Remember we need this to be less than the min of |W| and |Z|. Consider changing this as a hyperparameter

# ======================================================================================================================
#                                                NMF Parameters
# ======================================================================================================================
'''
Parameters for the NMF factorisation
nmf_method: Method for the NMF factorisation (currently uses https://github.com/bm424/mvcnmf/tree/master and activated by "volmin_2")
w_vol: Volume regularisation parameter
delta: Delta parameter for the NMF factorisation
n_iter: Number of iterations for the NMF factorisation
err_cut: Error cut-off for the NMF factorisation
'''
nmf_method = "volmin_2" 
w_vol = 0.01#0.1
delta = 1e-8
n_iter = 200000
err_cut = 1e-10

# ======================================================================================================================
#                                           Random Seed Initialisation
# ======================================================================================================================
np.random.seed(0)
torch.manual_seed(0)

In [3]:
# ======================================================================================================================
#                                          Matrices for Data Generation
# ======================================================================================================================
'''
These matrices are used to generate the data.
theta_w_epsilon: Matrix for W given Epsilon
theta_z_epsilon: Matrix for Z given Epsilon
theta_a_z_epsilon: Matrix for A given Z and Epsilon
theta_y_a_w_epsilon: Matrix for Y given A, W and Epsilon

The dimensions of the variables are as follows:
W: 2D
Z: 2D
A: 3D
Y: 2D
Epsilon: 2D
'''

theta_w_epsilon = torch.tensor([
    [-2, 2],
    [2, -2]
])

theta_z_epsilon = torch.tensor([
    [-2, 2],
    [2, -2]
])

theta_a_z_epsilon = torch.tensor([
    [2.0, -2.0],  
    [-2.0, 2.0],
    [2.0, -2.0]
]) 

# 1x5 matrix concatenating 1 hot of w and a. Negate when epsilon is 1. a is 3D and w is 2D
theta_y_a_w_epsilon = torch.tensor([
    [10, 3, -5, -3, 3]
]) 


In [4]:
# ======================================================================================================================
#                                          Data Generation Functions
# ======================================================================================================================

def get_tuple_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p=p_target):
    '''
    Generates a tuple of data points for the given parameters.

    Parameters:
    theta_w_epsilon: Matrix for W given Epsilon
    theta_z_epsilon: Matrix for Z given Epsilon
    theta_a_z_epsilon: Matrix for A given Z and Epsilon
    theta_y_a_w_epsilon: Matrix for Y given A, W and Epsilon
    p: Probability of a sample being from the target domain

    Returns:
    epsilon: Epsilon value via Bernoulli distribution
    z: Z value via Bernoulli distribution and sigmoid function
    w: W value via Bernoulli distribution and sigmoid function
    a: A value via theta_a_z_epsilon @ z or -theta_a_z_epsilon @ z depending on epsilon. One-hot encoded
    y: Y value via Bernoulli distribution and sigmoid function of theta_y_a_w_epsilon @ [w, a] or -theta_y_a_w_epsilon @ [w, a] depending on epsilon
    '''

    epsilon = torch.bernoulli(torch.tensor([p])).long() # (1,)
    epsilon_one_hot = torch.nn.functional.one_hot(epsilon.clone().detach(), num_classes=2).squeeze() # (1,2)

    z = torch.bernoulli(torch.sigmoid(epsilon_one_hot @ theta_z_epsilon)).float().squeeze()
    w = torch.bernoulli(torch.sigmoid(epsilon_one_hot @ theta_w_epsilon)).float().squeeze() # w given z and a should be perfect

    # w given z and a should be perfect

    a_logits = epsilon * theta_a_z_epsilon.float() @ z.float() + (epsilon-1) * theta_a_z_epsilon.float() @ z.float() #epsilon is 0 or 1, theta_a_z_epsilon is 3x2, z is [2]. So result is 3x1
    a_prob = torch.softmax(a_logits, dim=0)  # Apply softmax to convert logits to probabilities
    if debug:
        print(a_prob)   
    a_category = torch.multinomial(a_prob, 1).squeeze()  # Sample from the categorical distribution
    a = torch.nn.functional.one_hot(a_category, num_classes=3).float()  # Convert to one-hot encoding
    if debug:
        print(a)
    # a = torch.bernoulli(torch.sigmoid(a_logits)).float().squeeze() # a is size 3

    # concatenate w and a
    # then similar but use different matrix and w,a instead of z
    wa = torch.cat((w, a), dim=0) # [5]

    y_logits = epsilon * theta_y_a_w_epsilon @ wa.long() + (epsilon-1) * theta_y_a_w_epsilon @ wa.long() # y is a function of a, w and epsilon from the graph
    #print(torch.sigmoid(y_logits))
    y = torch.bernoulli(torch.sigmoid(y_logits).squeeze()).float()

    return epsilon,z,w,a,y


def get_dataset_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p, total):
    '''
    Generates a dataset of data points for the given parameters, using the get_tuple_new function.

    Parameters:
    theta_w_epsilon: Matrix for W given Epsilon
    theta_z_epsilon: Matrix for Z given Epsilon
    theta_a_z_epsilon: Matrix for A given Z and Epsilon
    theta_y_a_w_epsilon: Matrix for Y given A, W and Epsilon
    p: Probability of a sample being from the target domain
    total: Total number of samples

    Returns:
    U: List of Epsilon values
    Z: List of Z values
    W: List of W values
    X: List of A values
    Y: List of Y values
    '''

    U,Z,W,X,Y = [],[],[],[],[]
    for _ in range(total):
        u,z,w,x,y = get_tuple_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p)
        U.append(u)
        Z.append(z)
        W.append(w)
        X.append(x)
        Y.append(y)

    return U,Z,W,X,Y


def get_data_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p_target, total):
    '''
    Returns the source and target datasets for the given parameters.

    Parameters:
    theta_w_epsilon: Matrix for W given Epsilon
    theta_z_epsilon: Matrix for Z given Epsilon
    theta_a_z_epsilon: Matrix for A given Z and Epsilon
    theta_y_a_w_epsilon: Matrix for Y given A, W and Epsilon
    p_target: Probability of a sample being from the target domain
    total: Total number of samples

    Returns:
    (Z_source, U_source, W_source, X_source, Y_source): Source dataset
    (Z_target, U_target, W_target, X_target, Y_target): Target dataset
    '''
    
    # Source distribution data
    U_source, Z_source, W_source, X_source, Y_source = get_dataset_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, 1-p_target, total)
    
    # Target distribution data
    U_target, Z_target, W_target, X_target, Y_target = get_dataset_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p_target, total)
    
    return (Z_source, U_source, W_source, X_source, Y_source), \
           (Z_target, U_target, W_target, X_target, Y_target)

def tensor_to_binary(tensor):
    return 1 if tensor[0] == 1 else 0

def get_probabilities(model, Z_source, A_source):
    '''
    Function to get the probabilities of Y given all possible Z and A and put them in a 3D array.
    '''

    # Convert lists of tensors to numpy arrays
    Z = np.array([z.numpy() for z in Z_source])
    A = np.array([a.numpy() for a in A_source])
    
    num_Z = Z.shape[1]
    num_A = A.shape[1]

    # Generate all possible one-hot vectors for Z and A
    possible_Z = np.eye(num_Z)
    possible_A = np.eye(num_A)
    
    probabilities = []
    
    for z in possible_Z:
        for a in possible_A:
            ZA = np.hstack((z.reshape(1, -1), a.reshape(1, -1)))
            # Flatten ZA for the MLPClassifier
            ZA_flat = ZA.flatten().reshape(1, -1)
            prob = model.predict_proba(ZA_flat)[0]
            probabilities.append(prob)
    
    probabilities = np.array(probabilities).reshape((num_Z, num_A, 2))
    
    return probabilities

In [5]:
def run_algorithm(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p_source, p_target, total, specific_a_index, num_classes_Y, num_classes_W, num_features_Z, num_features_A, num_epsilon, nmf_method, w_vol, delta, n_iter, err_cut, summary = True):
    '''Runs the full algorithm'''

    # Generate the source and target datasets
    source_data, target_data = get_data_new(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p_target, total)
    Z_source, epsilon_source, W_source, A_source, Y_source = source_data
    Z_target, epsilon_target, W_target, A_target, Y_target = target_data

    # Split source data into training, validation, and test sets
    A_train_val_source, A_test_source, Z_train_val_source, Z_test_source, W_train_val_source, W_test_source, Y_train_val_source, Y_test_source, epsilon_train_val_source, epsilon_test_source = train_test_split(
    A_source, Z_source, W_source, Y_source, epsilon_source, test_size=0.2, random_state=42)
    A_train_source, A_val_source, Z_train_source, Z_val_source, W_train_source, W_val_source, Y_train_source, Y_val_source, epsilon_train_source, epsilon_val_source = train_test_split(
    A_train_val_source, Z_train_val_source, W_train_val_source, Y_train_val_source, epsilon_train_val_source, test_size=0.25, random_state=42)

    # Split target data into training, validation, and test sets
    A_train_val_target, A_test_target, Z_train_val_target, Z_test_target, W_train_val_target, W_test_target, Y_train_val_target, Y_test_target, epsilon_train_val_target, epsilon_test_target = train_test_split(
    A_target, Z_target, W_target, Y_target, epsilon_target, test_size=0.2, random_state=42)
    A_train_target, A_val_target, Z_train_target, Z_val_target, W_train_target, W_val_target, Y_train_target, Y_val_target, epsilon_train_target, epsilon_val_target = train_test_split(
    A_train_val_target, Z_train_val_target, W_train_val_target, Y_train_val_target, epsilon_train_val_target, test_size=0.25, random_state=42)

    # Step 1
    ZA_source = np.hstack((Z_train_source, A_train_source)) 
    model_Y = SklearnLogisticRegression(max_iter=1000)
    model_Y.fit(ZA_source, Y_train_source)
    Y_train_pred = model_Y.predict(ZA_source)
    Y_train_true = Y_train_source
    accuracy_Y_train = np.mean(Y_train_pred == Y_train_source)
    # test set
    ZA_source_test = np.hstack((Z_test_source, A_test_source))
    Y_test_pred = model_Y.predict(ZA_source_test)
    Y_test_true = Y_test_source
    accuracy_Y_test = np.mean(Y_test_pred == Y_test_true)
    # W 
    # Convert the list of tensors to a list of binary labels
    binary_labels = [tensor_to_binary(t) for t in W_train_source]

    # Convert to numpy array (optional)
    binary_labels_array = np.array(binary_labels)

    model_W= SklearnLogisticRegression(max_iter=1000)
    model_W.fit(ZA_source, binary_labels_array)
    W_train_pred = model_W.predict(ZA_source)
    W_train_true = W_source
    accuracy_W_train = np.mean(W_train_pred == binary_labels_array)
    # Test set
    W_test_pred = model_W.predict(ZA_source_test)
    W_test_true = np.array([tensor_to_binary(t) for t in W_test_source])
    accuracy_W_test = np.mean(W_test_pred == W_test_true)
    p_Y_given_ZA = get_probabilities(model_Y, Z_source, A_source)
    p_Y_given_ZA_matrix = p_Y_given_ZA[:, specific_a_index, :]
    p_W_given_ZA = get_probabilities(model_W, Z_source, A_source)
    p_W_given_ZA_matrix = p_W_given_ZA[:, specific_a_index, :]
    if debug:
        print("Step 1 done")

    # Step 2
    stacked_matrix = np.vstack((p_Y_given_ZA_matrix, p_W_given_ZA_matrix)) # this should be a |Y| x |Z| matrix stacked on top of a |W| x |Z| matrix (for specific a)
    W, H = mvc_nmf(stacked_matrix.T, num_epsilon, w_vol, n_iter, err_cut) # Transpose the matrix to match the input format of the function
    p_Y_given_epsilon = W[:num_classes_Y, :] # |Y| x |\Epsilon| matrix for specific a, the first num_classes_Y rows #CHECK NUM_CLASSES_Y IS THE ONE
    p_W_given_epsilon = W[num_classes_Y:, :] # |W| x |\Epsilon| matrix for specific a, the rest of the rows #CHECK NUM_CLASSES_Y IS THE ONE
    p_epsilon_given_ZA = H # |\Epsilon| x |Z| matrix for specific a
    reconstructed_stacked_matrix = np.dot(W, H)
    if debug:
        print("Step 2 done")

    # Step 3
    ZA_train_target = np.hstack((Z_train_target, A_train_target))
    # Convert the list of tensors to a list of binary labels
    binary_labels_target = [tensor_to_binary(t) for t in W_train_target]
    # Convert to numpy array (optional)
    binary_labels_target_array = np.array(binary_labels_target)
    model_q_W = SklearnLogisticRegression(max_iter=1000)
    model_q_W.fit(ZA_train_target, binary_labels_target_array)
    W_train_pred_target = model_q_W.predict(ZA_train_target)
    W_train_true_target = W_train_target
    accuracy_W_train_target = np.mean(W_train_pred_target == binary_labels_target_array)
    q_W_given_ZA = get_probabilities(model_q_W, Z_target, A_target)
    if debug:
        print("Step 3 done")

    # Step 4
    q_W_given_ZA_specific_a = q_W_given_ZA[:, specific_a_index, :]
    q_epsilon_given_Z_and_A, _, _, _ = np.linalg.lstsq(p_W_given_epsilon, q_W_given_ZA_specific_a, rcond=None) 
    q_epsilon_given_Z_and_A #note I'm getting a negative value here
    if debug:
        print("Step 4 done")

    # Step 5
    # Convert the list of tensors to a list of binary labels
    binary_labels_target_Z = [tensor_to_binary(t) for t in Z_train_target]
    # Convert to numpy array (optional)
    binary_labels_target_array_Z = np.array(binary_labels_target_Z)
    model_q_Z = SklearnLogisticRegression(max_iter=1000)#, class_weight = class_weights)
    model_q_Z.fit(A_train_target, binary_labels_target_array_Z)
    Z_train_pred_target = model_q_Z.predict(A_train_target)
    Z_train_true_target = Z_train_target
    accuracy_Z_train_target = np.mean(Z_train_pred_target == binary_labels_target_array_Z)
    if debug:
        print("Step 5 done")

    # Step 6
    one_hot_specific_a = np.eye(len(A_source[0]))[specific_a_index].reshape(1, -1)
    q_Z_given_A = model_q_Z.predict_proba(one_hot_specific_a)
    q_Y_given_A = p_Y_given_epsilon@(q_epsilon_given_Z_and_A@q_Z_given_A.T)
    q_Y_given_a_normalised = q_Y_given_A / np.sum(q_Y_given_A)
    if debug:
        print("Step 6 done")

    if summary:
        print("Summary:")
        print(f"Accuracy of model_Y on training set: {accuracy_Y_train * 100:.2f}%")
        print(f"Accuracy of model_Y on test set: {accuracy_Y_test * 100:.2f}%")
        print(f"Accuracy of model_W on training set: {accuracy_W_train * 100:.2f}%")
        print(f"Accuracy of model_W on test set: {accuracy_W_test * 100:.2f}%")
        print(f"Accuracy of model_W on training set (target): {accuracy_W_train_target * 100:.2f}%")
        print(f"Accuracy of model_W on test set (target): {accuracy_W_test * 100:.2f}%")
        print(f"Accuracy of model on training set (target): {accuracy_Z_train_target * 100:.2f}%")

    return q_Y_given_a_normalised

In [6]:
# Run algorithm for different values of A and p_target

In [7]:
p_target_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]  # Example values, you can define your own range
specific_a_index_values = [0, 1, 2]

In [8]:
for p_target in p_target_values:
    p_source = 1 - p_target
    for specific_a_index in specific_a_index_values:
        result = run_algorithm(theta_w_epsilon,theta_z_epsilon,theta_a_z_epsilon,theta_y_a_w_epsilon, p_source, p_target, total, specific_a_index, num_classes_Y, num_classes_W, num_features_Z, num_features_A, num_epsilon, nmf_method, w_vol, delta, n_iter, err_cut, summary = False)
        # Process the result as needed
        print(f"Result for p_target={p_target}, specific_a_index={specific_a_index}: {result}")

Result for p_target=0.1, specific_a_index=0: [[0.34939042]
 [0.64458964]]
Result for p_target=0.1, specific_a_index=1: [[0.50023259]
 [0.45286618]]
Result for p_target=0.1, specific_a_index=2: [[0.29635302]
 [0.60812819]]
Result for p_target=0.2, specific_a_index=0: [[0.31559285]
 [0.54537215]]
Result for p_target=0.2, specific_a_index=1: [[0.52111521]
 [0.46535484]]
Result for p_target=0.2, specific_a_index=2: [[0.32371805]
 [0.61364591]]
Result for p_target=0.3, specific_a_index=0: [[0.36231379]
 [0.49883146]]
Result for p_target=0.3, specific_a_index=1: [[0.51544238]
 [0.48656528]]
Result for p_target=0.3, specific_a_index=2: [[0.37602944]
 [0.59939354]]
Result for p_target=0.4, specific_a_index=0: [[0.41598081]
 [0.48127673]]
Result for p_target=0.4, specific_a_index=1: [[0.50170367]
 [0.49889235]]
Result for p_target=0.4, specific_a_index=2: [[0.44537426]
 [0.55072019]]
Result for p_target=0.5, specific_a_index=0: [[0.50590158]
 [0.50154415]]
Result for p_target=0.5, specific_a_in

KeyboardInterrupt: 